<a href="https://colab.research.google.com/github/OhJin-Soo/tensorflow/blob/main/deep_%26_wide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

## Functional API

In [ ]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [ ]:
input_wide = tf.keras.layers.Input(shape=[5])  # 특성 0 ~ 4
input_deep = tf.keras.layers.Input(shape=[6])  # 특성 2 ~ 7
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])

In [ ]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - dense_2_RootMeanSquaredError: 1.5791 - dense_2_loss: 2.6061 - dense_3_RootMeanSquaredError: 1.9193 - dense_3_loss: 3.7692 - loss: 2.7225 - val_dense_2_RootMeanSquaredError: 0.8511 - val_dense_2_loss: 0.7241 - val_dense_3_RootMeanSquaredError: 3.4449 - val_dense_3_loss: 11.8617 - val_loss: 1.8386
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_2_RootMeanSquaredError: 0.7249 - dense_2_loss: 0.5261 - dense_3_RootMeanSquaredError: 0.9657 - dense_3_loss: 0.9335 - loss: 0.5669 - val_dense_2_RootMeanSquaredError: 0.6540 - val_dense_2_loss: 0.4276 - val_dense_3_RootMeanSquaredError: 2.3411 - val_dense_3_loss: 5.4781 - val_loss: 0.9330
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_2_RootMeanSquaredError: 0.6714 - dense_2_loss: 0.4511 - dense_3_RootMeanSquaredError: 0.7915 - dense_3_loss: 0.6266 - loss: 0.4686 - val_dense_2_RootMeanSquaredError: 0.6188 - val_dense_2_loss: 0.3828 - val_dense_3_RootMeanSquaredError: 1

In [ ]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_rmse, aux_rmse = eval_results[:3]

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - dense_2_RootMeanSquaredError: 0.5883 - dense_2_loss: 0.3462 - dense_3_RootMeanSquaredError: 0.6369 - dense_3_loss: 0.4058 - loss: 0.3521


In [ ]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step


In [ ]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


## Subclassing API

In [ ]:
import keras

@keras.saving.register_keras_serializable()
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

tf.random.set_seed(42)
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - RootMeanSquaredError: 1.6634 - RootMeanSquaredError_1: 1.7407 - loss: 2.9073 - mse_loss: 3.1157 - val_RootMeanSquaredError: 1.1397 - val_RootMeanSquaredError_1: 1.3322 - val_loss: 1.3464 - val_mse_loss: 1.7741
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.7662 - RootMeanSquaredError_1: 0.9027 - loss: 0.6105 - mse_loss: 0.8162 - val_RootMeanSquaredError: 0.7398 - val_RootMeanSquaredError_1: 0.8071 - val_loss: 0.5577 - val_mse_loss: 0.6514
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6887 - RootMeanSquaredError_1: 0.7835 - loss: 0.4885 - mse_loss: 0.6140 - val_RootMeanSquaredError: 0.6397 - val_RootMeanSquaredError_1: 0.7538 - val_loss: 0.4252 - val_mse_loss: 0.5681
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6585 - RootMeanSquaredError_1: 0.7505 - loss: 0.4468 - mse_loss: 0.5634 - val_RootMeanSquaredError: 0.6210 - val_RootMeanSquaredEr

In [ ]:
import shutil

shutil.rmtree("my_keras_model", ignore_errors=True)

In [ ]:
model.save("my_keras_model.keras")

In [ ]:
model = tf.keras.models.load_model("my_keras_model.keras")
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step


In [ ]:
model.save_weights("my_weights.weights.h5")

In [ ]:
model.load_weights("my_weights.weights.h5")

In [ ]:
model.save("my_model.keras")

In [ ]:
shutil.rmtree("my_checkpoints", ignore_errors=True)

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints.weights.h5",
                                                   save_weights_only=True)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[checkpoint_cb])

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - RootMeanSquaredError: 0.6046 - RootMeanSquaredError_1: 0.6752 - loss: 0.3747 - mse_loss: 0.4559 - val_RootMeanSquaredError: 1.1359 - val_RootMeanSquaredError_1: 1.0161 - val_loss: 1.2644 - val_mse_loss: 1.0321
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6001 - RootMeanSquaredError_1: 0.6658 - loss: 0.3685 - mse_loss: 0.4434 - val_RootMeanSquaredError: 1.2808 - val_RootMeanSquaredError_1: 1.5725 - val_loss: 1.7238 - val_mse_loss: 2.4718
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5965 - RootMeanSquaredError_1: 0.6612 - loss: 0.3640 - mse_loss: 0.4373 - val_RootMeanSquaredError: 0.9629 - val_RootMeanSquaredError_1: 0.9258 - val_loss: 0.9203 - val_mse_loss: 0.8569
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5916 - RootMeanSquaredError_1: 0.6535 - loss: 0.3578 - mse_loss: 0.4272 - val_RootMeanSquaredError: 1.0625 - val_RootMeanSquaredE

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=100,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.5749 - RootMeanSquaredError_1: 0.6270 - loss: 0.3369 - mse_loss: 0.3932 - val_RootMeanSquaredError: 0.6169 - val_RootMeanSquaredError_1: 0.6381 - val_loss: 0.3832 - val_mse_loss: 0.4071
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5723 - RootMeanSquaredError_1: 0.6245 - loss: 0.3338 - mse_loss: 0.3900 - val_RootMeanSquaredError: 0.8794 - val_RootMeanSquaredError_1: 1.0350 - val_loss: 0.8031 - val_mse_loss: 1.0707
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5704 - RootMeanSquaredError_1: 0.6221 - loss: 0.3316 - mse_loss: 0.3871 - val_RootMeanSquaredError: 0.6743 - val_RootMeanSquaredError_1: 0.6951 - val_loss: 0.4575 - val_mse_loss: 0.4831
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - RootMeanSquaredError: 0.5694 - RootMeanSquaredError_1: 0.6209 - loss: 0.3305 - mse_loss: 0.3856 - val_RootMeanSquaredError: 1.0960 - val_RootMeanSquar

In [ ]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        ratio = logs["val_loss"] / logs["loss"]
        print(f"Epoch={epoch}, val/train={ratio:.2f}")

In [ ]:
val_train_ratio_cb = PrintValTrainRatioCallback()
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[val_train_ratio_cb], verbose=0)

Epoch=0, val/train=1.60
Epoch=1, val/train=1.66
Epoch=2, val/train=3.69
Epoch=3, val/train=1.44
Epoch=4, val/train=2.46
Epoch=5, val/train=1.15
Epoch=6, val/train=2.20
Epoch=7, val/train=1.36
Epoch=8, val/train=1.84
Epoch=9, val/train=0.98
